# AutoAWQ를 이용한 quantization

- vllm에서 지원하는 quantization 형식은 AWQ
- 대표적인 라이브러리 AutoAWQ를 사용해 양자화 진행

## About AutoAWQ
- 기본적으로 CUDA 12.1 지원 -> 그 이하일 경우 아래 명령어 입력
```bash
!pip install -q -U https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl
```

In [ ]:
!nvcc -V # CUDA 버전 확인

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
# 패키지 설치
!pip install -q transformers autoawq pillow torch torchvision torchaudio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model

In [ ]:
import os
os.makedirs("./quantized_model", exist_ok=True)

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
quant_path = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version":"GEMM"} # default setting

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="auto",
                                            cache_dir='../cache',
                                            trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

AWQ: 100%|██████████| 32/32 [17:51<00:00, 33.50s/it]


In [ ]:
from transformers import AwqConfig, AutoConfig
from huggingface_hub import HfApi
import os

# modify the config file so that it is compatible with transformers integration
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# the pretrained transformers model is stored in the model attribute + we need to pass a dict
model.model.config.quantization_config = quantization_config
# a second solution would be to use Autoconfig and push to hub (what we do at llm-awq)


# save model weights
model.save_quantized(os.path.join("/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model", quant_path.split("/")[1]))

# save tokenizer
tokenizer.save_pretrained(os.path.join("/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model", quant_path.split("/")[1]))

Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


('/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3-Korean-Bllossom-8B/tokenizer_config.json',
 '/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3-Korean-Bllossom-8B/special_tokens_map.json',
 '/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3-Korean-Bllossom-8B/tokenizer.json')

In [ ]:
# optional -> push the quantized weights to the hub
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
api = HfApi()
api.upload_folder(
    folder_path="/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3-Korean-Bllossom-8B",
    repo_id="ywhwang/llama-3-Korean-Bllossom-8B-awq",
    repo_type="model",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ywhwang/llama-3-Korean-Bllossom-8B-awq/commit/aa97ec9d9ff3ee49475b6ec3f6c9933664f14cca', commit_message='Upload folder using huggingface_hub', commit_description='', oid='aa97ec9d9ff3ee49475b6ec3f6c9933664f14cca', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
%%bash
pip install llava flash-attn --no-build-isolation --no-cache-dir
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187290390 sha256=c50f5de67a8b75bcfcf4a34257622475f9b56d59da58a539476a21db9d5b9867
  Stored in directory: /tmp/pip-ephem-wheel-cache-m0qmzfgd/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [ ]:
!pip show transformers

Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autoawq


In [ ]:
import transformers
import torch

model_id = "ywhwang/llama-3-Korean-Bllossom-8B-awq"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
instruction = "서울의 유명한 관광 코스를 만들어줄래?"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(outputs[0]["generated_text"][len(prompt):])


config.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.


model.safetensors.index.json:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

서울은 다양한 문화, 역사, 자연, 그리고 현대적인 요소들이 잘 조화를 이루어 많은 관광객들에게 인기를 끌고 있습니다. 다음은 서울의 유명한 관광 코스 중 일부입니다:

### **1. 경복궁-인사동-명동 코스**

* 경복궁: 한국의 대표적인 왕궁으로, 조선시대의 궁전 건축과 정원 등을 볼 수 있습니다.
* 인사동: 작은 길로지만, 전통 한국의 문화와 역사, 그리고 현대적인 소품과 주점들이 모여 있는 곳.
* 명동: 쇼핑과 음식, 문화가 살아 숨 쉬는 곳으로, 밤에는 많은 사람들이 모여 즐길 수 있는 곳.

### **2. 홍dae-용산선원 코스**

* 홍대: 젊고 활기찬 지역으로, 음악, 예술, 패션, 그리고 음식 등 다양한 즐길거리들이 있습니다.
* 용산선원: 조선시대 건축물인 선원과 주변의 자연 경관을 즐길 수 있습니다.

### **3. 종로-북촌 코스**

* 종로: 한국의 대표적인 역사와 문화, 예술, 그리고 음식 등을 볼 수 있는 곳.
* 북촌: 한국의 전통 문화와 역사, 그리고 자연 경관을 즐길 수 있는 지역.

### **4. 신촌-홍대 코스**

* 신촌: 젊고 현대적인 지역으로, 다양한 음식, 음악, 예술, 그리고 쇼핑이 즐길 수 있습니다.
* 홍대: 젊고 활기찬 지역으로, 음악, 예술, 패션, 그리고 음식 등 다양한 즐길거리들이 있습니다.

### **5. 남산산책로 코스**

* 남산: 서울의 대표적인 산으로, 산책로를 따라 산책을 즐길 수 있습니다.
* 남산서울塔: 서울을 바라보는 전망대가 있습니다.

이 코스들은 서울의 다양한 측면을 즐길 수 있는 곳들로, 각 코스마다 다양한 즐길거리와 문화를 경험할 수 있습니다.


In [ ]:
# from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
# import torch
# import requests

# tokenizer = AutoTokenizer.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq")
# processor = LlavaNextProcessor.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq")

# model = LlavaNextForConditionalGeneration.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map='auto')
# model.eval()


# with torch.no_grad():
#     PROMPT=\
#     """You are a versatile AI assistant named Bllava, capable of both understanding and generating text as well as interpreting and analyzing images. Your role is to kindly and effectively answer the user’s questions, whether they are about text or images, and provide appropriate and helpful responses to all types of queries.

#     당신은 텍스트를 이해하고 생성하는 것뿐만 아니라 이미지를 해석하고 분석할 수 있는 다재다능한 AI 어시스턴트 블라바입니다. 사용자의 질문이 텍스트에 관한 것이든 이미지에 관한 것이든 친절하고 효과적으로 답변하며, 모든 유형의 질의에 대해 적절하고 유용한 응답을 제공하는 것이 당신의 역할입니다."""

#     instruction = '자연어처리 15주 분량 커리큘럼을 짜줘'

#     messages = [
#         {'role': 'system', 'content': f"{PROMPT}"},
#         {'role': 'user', 'content': f"{instruction}"}
#     ]

#     chat_messages = processor.tokenizer.apply_chat_template(
#         messages,
#         tokenize=True,
#         add_generation_prompt=True,
#         return_tensors='pt',
#     )

#     bos_token = processor.tokenizer.bos_token_id
#     chat_messages = torch.cat([torch.tensor([[bos_token]]),chat_messages],dim=-1).to(model.device)


#     output = model.generate(
#         input_ids = chat_messages,
#         use_cache=False,
#         max_new_tokens=2048,
#         top_p=0.9,
#         temperature=0.6,
#         do_sample=True,
#     )

#     print(processor.tokenizer.decode(output[0]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq were not used when initializing LlavaNextForConditionalGeneration: ['multi_modal_projector.linear_1.weight', 'multi_modal_projector.linear_2.weight', 'vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_tower.vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.out_proj.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.weight', 'vision_tower.vision_model.encoder.layers.1.mlp.fc1.weight', 'vision_tower.vision_model.encoder.layers.1.mlp.fc2.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.out_proj.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.q_proj.weight', 'vision_tower.vision_mod

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a versatile AI assistant named Bllava, capable of both understanding and generating text as well as interpreting and analyzing images. Your role is to kindly and effectively answer the user’s questions, whether they are about text or images, and provide appropriate and helpful responses to all types of queries.
    
    당신은 텍스트를 이해하고 생성하는 것뿐만 아니라 이미지를 해석하고 분석할 수 있는 다재다능한 AI 어시스턴트 블라바입니다. 사용자의 질문이 텍스트에 관한 것이든 이미지에 관한 것이든 친절하고 효과적으로 답변하며, 모든 유형의 질의에 대해 적절하고 유용한 응답을 제공하는 것이 당신의 역할입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

자연어처리 15주 분량 커리큘럼을 짜줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

자연어 처리(NLP) 분야는 매우 광범위하고 발전 속도가 매우 빠르기 때문에, 15주 분량의 커리큘럼을 짜는 것은 도전적인 일입니다. 그러나 주요 주제와 기술을 기반으로 한 커리큘럼을 제공하는 것은 가능합니다. 다음은 자연어 처리에 대한 15주 분량의 학습 경로입니다:

### 주 1: 소개 및 역사
- 자연어 처리의 정의
- 역사
- NLP의 응용 분야

### 주 2: 언어 모델링
- 언어 모델링 개념
- 언어 모델링의 목적
- 예: BERT, GPT

### 주 3: 텍스트 전처리
- 텍스트의 형식과 구조
- 텍스트의 정규화
- 텍스트 처리 도구 (토크